In [ ]:
import pandas as pd
import re
from collections import defaultdict
import nltk
from nltk import pos_tag, word_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

In [11]:
def extract_important_words(text):
    words = word_tokenize(text)
    tagged_words = pos_tag(words)
    important_words = [word.lower().strip() for word, tag in tagged_words if tag.startswith('NN') or tag.startswith('JJ')]
    return important_words

In [12]:
def assign_ids(column):
    unique_values = {}
    counter = 1
    ids = []

    for value in column:
        if value not in unique_values:
            unique_values[value] = counter
            counter += 1
        ids.append(unique_values[value])

    return ids, unique_values

In [19]:
file_path = '/content/drive/MyDrive/Datasets Normalization/Netflix Reviews/netflix_reviews.csv'
df = pd.read_csv(file_path)

user_ids = {name: idx for idx, name in enumerate(df['userName'].unique(), 1)}
df['userId'] = df['userName'].map(user_ids)

cols = df.columns.tolist()
cols.insert(cols.index('userName'), cols.pop(cols.index('userId')))
df = df[cols]

word_to_id = defaultdict(lambda: len(word_to_id) + 1)
df['review'] = df['content'].apply(lambda x: ', '.join(extract_important_words(str(x))) if isinstance(x, str) else '')
df['reviewId'] = df['review'].apply(lambda x: ', '.join([str(word_to_id[word]) for word in x.split(', ')]))

cols = df.columns.tolist()
cols.insert(cols.index('score'), cols.pop(cols.index('review')))
df = df[cols]

In [20]:
df.dropna(how='all', inplace=True)
df.dropna(axis=1, how='all', inplace=True)

In [21]:
output_file_path = '/content/drive/MyDrive/Datasets Normalization/Netflix Reviews/processed_netflix_reviews.csv'
df.to_csv(output_file_path, index=False)

print('Done!')

Done!
